# Восстановление золота из руды

Необходимо подготовить прототип модели машинного обучения для компании «Цифры», которая разрабатывает решения для эффективной работы промышленных предприятий.
Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. У нас есть данные с параметрами добычи и очистки.
Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.

В процессе выполнения проекта необходимо:

- Подготовить данные;
- Провести исследовательский анализ данных;
- Построить и обучить модель.

***Технологический процесс***

Когда добытая руда проходит первичную обработку, получается дроблёная смесь. Её отправляют на флотацию (обогащение) и двухэтапную очистку.


**Стадии:**

1. Флотация

Во флотационную установку подаётся смесь золотосодержащей руды. После обогащения получается черновой концентрат и «отвальные хвосты», то есть остатки продукта с низкой концентрацией ценных металлов.
На стабильность этого процесса влияет непостоянное и неоптимальное физико-химическое состояние флотационной пульпы (смеси твёрдых частиц и жидкости).

2. Очистка

Черновой концентрат проходит две очистки. На выходе получается финальный концентрат и новые отвальные хвосты.


***Описание данных***

**Технологический процесс**

* `Rougher feed` — исходное сырье
* `Rougher additions` (или `reagent additions`) — флотационные реагенты: Xanthate, Sulphate, Depressant
    * `Xanthat **` — ксантогенат (промотер, или активатор флотации);
    * `Sulphate` — сульфат (на данном производстве сульфид натрия);
    * `Depressant` — депрессант (силикат натрия).
* `Rougher process` (англ. «грубый процесс») — флотация
* `Rougher tails` — отвальные хвосты
* `Float banks` — флотационная установка
* `Cleaner process` — очистка
* `Rougher Au` — черновой концентрат золота
* `Final Au` — финальный концентрат золота

**Параметры этапов**

* `air amount` — объём воздуха
* `fluid levels` — уровень жидкости
* `feed size` — размер гранул сырья
* `feed rate` — скорость подачи


Нужно спрогнозировать сразу две величины:

1. эффективность обогащения чернового концентрата `rougher.output.recovery`;
2. эффективность обогащения финального концентрата `final.output.recovery`.


Данные находятся в трёх файлах:

* `gold_recovery_train_new_8ds.csv` — обучающая выборка;
* `gold_recovery_test_new_8ds.csv` — тестовая выборка;
* `gold_recovery_full_new_8ds.csv` — исходные данные.

Данные индексируются датой и временем получения информации (признак `date`). Соседние по времени параметры часто похожи.

Некоторые параметры недоступны, потому что замеряются и/или рассчитываются значительно позже. Из-за этого в тестовой выборке отсутствуют некоторые признаки, которые могут быть в обучающей. Также в тестовом наборе нет целевых признаков.

Исходный датасет содержит обучающую и тестовую выборки со всеми признаками.

Данные сырые. Необхоима их проверка и предобработка.

## Загрузка и подготовка данных

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important; }</style>"))

***Выгрузка библиотек***

In [29]:
import pandas as pd
pd.set_option('display.max_columns', None)

***Константы***

In [4]:
RANDOM_SEED = 0

In [5]:
# Based on: https://stackoverflow.com/a/17303428 (для красивого оформления текста)
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

***Функции***

In [6]:
#Рандоманая выгрузка строк
show_sample = lambda x: display(x.sample(5, random_state=RANDOM_SEED).reset_index(drop=True))

In [26]:
def com_info(data):
    name = [x for x in globals() if globals()[x] is data][0]
    print(color.UNDERLINE + 'Общая информация по данным '+ name + color.END)
    show_sample(data)
    print(data.info())
    display(pd.DataFrame(round((data.isna().mean()), 2)).style.background_gradient('coolwarm')\
                                                                .set_caption('% пропусков').format({0:'{:.2%}'.format}))
    display(data.describe().T.style.set_caption('Статистика'))
    print('Количество дублей: ', data.duplicated().sum())
    print('******************************************************************')
    print('')

***Загрука данных***

In [27]:
df_train = pd.read_csv('../datasets/gold_recovery_train_new_8ds.csv')
df_test = pd.read_csv('../datasets/gold_recovery_test_new_8ds.csv')
df_full = pd.read_csv('../datasets/gold_recovery_full_new_8ds.csv')

## Изучение общей информации

In [28]:
df_list = [df_train, df_test, df_full]
for i in df_list:
    com_info(i)

Общая информация по данным df_train


,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2018-06-08 12:59:59,4.068389,11.448127,5.983031,45.728369,63.185926,7.436988,2.577877,10.211517,2.042844,...,30.026765,-485.043326,22.022924,-466.815927,25.026081,-495.662399,21.919914,-500.075170,24.989364,-501.420340
1,2016-06-24 08:59:59,4.672297,10.013952,8.417600,45.142223,70.512351,8.217110,1.566327,11.871072,2.433252,...,15.997023,-499.721909,7.967451,-501.191492,16.024186,-500.172937,7.020786,-500.459658,24.969568,-498.972262
2,2017-03-12 03:59:59,4.999209,8.771206,8.396900,48.307998,66.501485,9.825832,3.145526,7.156999,3.558258,...,25.012867,-400.070250,22.977935,-400.084529,20.060910,-449.437364,19.983941,-449.840148,24.992126,-499.687118
3,2016-06-08 04:59:59,3.879375,9.816033,7.747834,47.631350,49.304890,8.156908,2.203170,9.323327,3.720784,...,8.983344,-550.003505,7.115827,-550.971423,7.866452,-551.177450,6.040703,-550.015381,18.010314,-500.538411
4,2016-08-01 14:59:59,4.403698,10.823148,9.911247,43.927370,69.502636,9.344400,2.947228,12.024562,3.293919,...,20.074004,-401.480443,10.031345,-402.416679,10.010939,-400.062267,4.957975,-400.499557,23.045125,-500.498783


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14149 entries, 0 to 14148
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                14149 non-null  object 
 1   final.output.concentrate_ag                         14148 non-null  float64
 2   final.output.concentrate_pb                         14148 non-null  float64
 3   final.output.concentrate_sol                        13938 non-null  float64
 4   final.output.concentrate_au                         14149 non-null  float64
 5   final.output.recovery                               14149 non-null  float64
 6   final.output.tail_ag                                14149 non-null  float64
 7   final.output.tail_pb                                14049 non-null  float64
 8   final.output.tail_sol                               14144 non-null  float64


,0
date,0.00%
final.output.concentrate_ag,0.00%
final.output.concentrate_pb,0.00%
final.output.concentrate_sol,1.00%
final.output.concentrate_au,0.00%
final.output.recovery,0.00%
final.output.tail_ag,0.00%
final.output.tail_pb,1.00%
final.output.tail_sol,0.00%
final.output.tail_au,0.00%


,count,mean,std,min,25%,50%,75%,max
final.output.concentrate_ag,14148.000000,5.142034,1.369586,0.000000,4.211620,4.994652,5.859540,16.001945
final.output.concentrate_pb,14148.000000,10.132960,1.654930,0.000000,9.297355,10.297144,11.170603,17.031899
final.output.concentrate_sol,13938.000000,9.202849,2.790516,0.000000,7.484645,8.845462,10.487508,18.124851
final.output.concentrate_au,14149.000000,44.003792,4.905261,0.000000,43.276111,44.872436,46.166425,52.756638
final.output.recovery,14149.000000,66.518832,10.295402,0.000000,62.545817,67.432775,72.346428,100.000000
final.output.tail_ag,14149.000000,9.607035,2.319069,0.000000,7.997429,9.480270,11.003707,19.552149
final.output.tail_pb,14049.000000,2.597298,0.971843,0.000000,1.905973,2.592022,3.241723,5.639565
final.output.tail_sol,14144.000000,10.512122,3.003617,0.000000,8.811324,10.514621,11.933009,22.317730
final.output.tail_au,14149.000000,2.918421,0.903712,0.000000,2.368607,2.851025,3.434764,8.197408
primary_cleaner.input.sulfate,14129.000000,133.320659,39.431659,0.003112,107.006651,133.018328,159.825396,250.127834


...

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2017-02-08 14:59:59,5.295988,8.195301,13.379583,47.758663,65.280176,9.925318,3.387088,7.605304,3.382011,...,24.937716,-499.827034,22.955855,-499.981726,23.018173,-499.465128,20.014626,-500.003639,24.991929,-599.773615
1,2016-10-29 13:59:59,3.020476,9.071122,16.816693,49.797229,63.724514,8.165338,3.167601,10.705567,3.591239,...,19.999104,-500.526123,16.899111,-500.418689,17.000241,-497.845707,13.990843,-499.834974,0.127304,-808.800767
2,2017-08-10 06:59:59,3.455687,12.662607,6.890618,45.799572,62.002449,7.103502,4.485478,11.579938,3.446024,...,16.031300,-500.548355,12.014055,-399.804953,15.987138,-500.414523,11.990190,-500.074479,15.009936,-500.026179
3,2018-03-20 17:59:59,5.360341,12.297913,6.589168,42.586163,56.365844,8.325812,1.211108,8.217830,2.221499,...,23.051530,-500.327972,14.984485,-499.893529,30.004056,-499.853016,15.031900,-500.068586,12.989058,-499.878706
4,2017-02-26 19:59:59,4.645203,10.322894,14.283728,46.100397,74.086821,12.162428,4.429356,9.392597,3.720634,...,24.994581,-398.620115,23.010430,-399.770903,22.975057,-455.682049,20.005831,-449.548036,25.005151,-499.842326


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19439 entries, 0 to 19438
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                19439 non-null  object 
 1   final.output.concentrate_ag                         19438 non-null  float64
 2   final.output.concentrate_pb                         19438 non-null  float64
 3   final.output.concentrate_sol                        19228 non-null  float64
 4   final.output.concentrate_au                         19439 non-null  float64
 5   final.output.recovery                               19439 non-null  float64
 6   final.output.tail_ag                                19438 non-null  float64
 7   final.output.tail_pb                                19338 non-null  float64
 8   final.output.tail_sol                               19433 non-null  float64


,0
date,0.00%
final.output.concentrate_ag,0.00%
final.output.concentrate_pb,0.00%
final.output.concentrate_sol,1.00%
final.output.concentrate_au,0.00%
final.output.recovery,0.00%
final.output.tail_ag,0.00%
final.output.tail_pb,1.00%
final.output.tail_sol,0.00%
final.output.tail_au,0.00%


,count,mean,std,min,25%,50%,75%,max
final.output.concentrate_ag,19438.000000,5.168470,1.372348,0.000000,4.251240,5.066094,5.895527,16.001945
final.output.concentrate_pb,19438.000000,9.978895,1.669240,0.000000,9.137262,10.102433,11.035769,17.031899
final.output.concentrate_sol,19228.000000,9.501224,2.787537,0.000000,7.722820,9.218961,10.947813,19.615720
final.output.concentrate_au,19439.000000,44.076513,5.129784,0.000000,43.402215,45.011244,46.275313,52.756638
final.output.recovery,19439.000000,67.050208,10.125840,0.000000,63.299712,68.172738,72.686642,100.000000
final.output.tail_ag,19438.000000,9.688589,2.328642,0.000000,8.062878,9.743623,11.134294,19.552149
final.output.tail_pb,19338.000000,2.705795,0.949077,0.000000,2.040119,2.748730,3.333216,5.804178
final.output.tail_sol,19433.000000,10.583728,2.868782,0.000000,8.938947,10.622456,12.104271,22.317730
final.output.tail_au,19439.000000,3.042467,0.922808,0.000000,2.461138,2.984909,3.571351,8.245022
primary_cleaner.input.sulfate,19415.000000,144.624774,44.464071,0.003112,114.107029,143.232103,175.075656,265.983123


Количество дублей:  0
******************************************************************



***df_train***

- Типы данных не оптимизированы, необходимо понизить разрядность признаков
- В данных есть пропуски, больше всего в признаке `secondary_cleaner.output.tail_sol` (11%)
- Некоторые признаки отрицательные - например, `primary_cleaner.state.floatbank8_b_level`, `primary_cleaner.state.floatbank8_c_level`, `rougher.state.floatbank10_a_level` и т.д.
- Полных дублей в данных нет.

## Предобработка